<a href="https://colab.research.google.com/github/Yshen-11/DUKE_XAI/blob/main/XAI_Assignment_2_Adversarial_Patches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision

In [2]:
import os
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from tqdm import tqdm
import numpy as np
from PIL import Image, ImageDraw, ImageOps
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import requests
## PyTorch
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10


In [3]:
from google.colab import files
uploaded = files.upload()  # Upload your image here

# Extract the filename
test_image = list(uploaded.keys())[0]
print(f"Uploaded image: {test_image}")


Saving testimage_panda.jpg to testimage_panda.jpg
Uploaded image: testimage_panda.jpg


In [4]:
# Load the ResNet34 model pre-trained on ImageNet
model = models.resnet34(weights='IMAGENET1K_V1')
model.eval()  # Set model to evaluation mode

# Download and load the class labels for ImageNet
imagenet_classes = requests.get('https://raw.githubusercontent.com/AIPI-590-XAI/Duke-AI-XAI/main/adversarial-ai-example-notebooks/imagenet_classes.txt').text.splitlines()

# Helper function to preprocess the image
def preprocess(image_path):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    return image

# Function to classify the image
def classify_image(image_path):
    image = preprocess(image_path)
    outputs = model(image)
    _, predicted = outputs.max(1)
    predicted_class = imagenet_classes[predicted.item()]  # Match index to class name
    return predicted_class

# Test with the uploaded image
predicted_class = classify_image(test_image)

# Print the predicted class name
print(f"Predicted class: {predicted_class}")

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 72.4MB/s]


Predicted class: giant panda


In [6]:
import os
import urllib.request
from urllib.error import HTTPError
import zipfile

# Define the paths
DATASET_PATH = "./dataset"  # You can change this to where you want to store the dataset
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial10/"

# Only the dataset file to download
dataset_file = "TinyImageNet.zip"
dataset_url = base_url + dataset_file

# Create the dataset directory if it doesn't exist
os.makedirs(DATASET_PATH, exist_ok=True)

# Full file path
file_path = os.path.join(DATASET_PATH, dataset_file)

# Check if the dataset file already exists, if not download it
if not os.path.isfile(file_path):
    print(f"Downloading {dataset_url}...")
    try:
        urllib.request.urlretrieve(dataset_url, file_path)
    except HTTPError as e:
        print("Something went wrong. Please try to download the file from the GDrive folder, or contact the author with the full output including the following error:\n", e)

# Unzip the dataset if it is a zip file
if dataset_file.endswith(".zip"):
    print("Unzipping file...")
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(DATASET_PATH)

print("Download and extraction completed!")

Unzipping file...
Download and extraction completed!


In [7]:
# Mean and Std from ImageNet
NORM_MEAN = np.array([0.485, 0.456, 0.406])
NORM_STD = np.array([0.229, 0.224, 0.225])
# No resizing and center crop necessary as images are already preprocessed.
plain_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN,
                         std=NORM_STD)
])

# Load dataset and create data loader
imagenet_path = os.path.join(DATASET_PATH, "TinyImageNet/")
assert os.path.isdir(imagenet_path), f"Could not find the ImageNet dataset at expected path \"{imagenet_path}\". " + \
                                     f"Please make sure to have downloaded the ImageNet dataset here, or change the {DATASET_PATH=} variable."
dataset = torchvision.datasets.ImageFolder(root=imagenet_path, transform=plain_transforms)

In [8]:
# Load training dataset
train_set, val_set = torch.utils.data.random_split(dataset, [4500, 500])
train_loader = data.DataLoader(train_set, batch_size=32, shuffle=True, drop_last=True, num_workers=8)

# Assuming imagenet_classes is already loaded
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(imagenet_classes)}

def place_patch(img, patch):
    for i in range(img.shape[0]):
        h_offset = np.random.randint(0,img.shape[2]-patch.shape[1]-1)
        w_offset = np.random.randint(0,img.shape[3]-patch.shape[2]-1)
        img[i,:,h_offset:h_offset+patch.shape[1],w_offset:w_offset+patch.shape[2]] = patch_forward(patch)
    return img

TENSOR_MEANS, TENSOR_STD = torch.FloatTensor(NORM_MEAN)[:,None,None], torch.FloatTensor(NORM_STD)[:,None,None]
def patch_forward(patch):
    # Map patch values from [-infty,infty] to ImageNet min and max
    patch = (torch.tanh(patch) + 1 - 2 * TENSOR_MEANS) / (2 * TENSOR_STD)
    return patch

def generate_adversarial_patch(model, target_class_name, patch_size, num_epochs=5):
   # Map class name to index
    if target_class_name in class_to_idx:
        target_class = class_to_idx[target_class_name]
    else:
        raise ValueError(f"Class {target_class_name} not found in ImageNet classes.")

    # Initialize patch parameters
    if not isinstance(patch_size, tuple):
        patch_size = (patch_size, patch_size)
    patch = nn.Parameter(torch.zeros(3, patch_size[0], patch_size[1]), requires_grad=True)

    # Define optimizer and loss function
    optimizer = optim.SGD([patch], lr=1e-1, momentum=0.8)
    loss_module = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(num_epochs):
        # Assuming train_loader and dataset already exist, the training set contains input images
        t = tqdm(train_loader, leave=False)
        for img, _ in t:
            img = place_patch(img, patch)
            img = img.to(device)
            pred = model(img)
            labels = torch.zeros(img.shape[0], device=pred.device, dtype=torch.long).fill_(target_class)
            loss = loss_module(pred, labels)

            # Gradient update
            optimizer.zero_grad()
            loss.mean().backward()
            optimizer.step()
            t.set_description(f"Epoch {epoch}, Loss: {loss.item():4.2f}")

    # Save the patch as an image
    save_patch_as_image(patch.data, 'adversarial_patch.jpg')

    # Return the generated patch
    return patch.data

def save_patch_as_image(patch_tensor, filename):
    # Convert the patch tensor into a savable format (e.g., from tensor to image)
    patch_np = patch_tensor.detach().cpu().numpy()
    patch_np = (patch_np - patch_np.min()) / (patch_np.max() - patch_np.min())  # 归一化到0-1之间
    patch_np = (patch_np * 255).astype(np.uint8)  # Convert to 8-bit integer

    # Dimension transformation, from (C, H, W) to (H, W, C)
    patch_np = np.transpose(patch_np, (1, 2, 0))

    # Use PIL library to save the patch as a JPEG format image
    patch_img = Image.fromarray(patch_np)
    patch_img.save(filename)
    print(f"Adversarial patch saved as {filename}")

# Usage example
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
target_model = model  # Replace with the model you want to attack
target_class_name = 'violin'  # Input class name from ImageNet
patch_size = 96  # Size of the patch
patch = generate_adversarial_patch(target_model, target_class_name, patch_size)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


KeyboardInterrupt: 

In [26]:
patched_uploaded = files.upload()  # Upload your image here

# Extract the filename
patched_test_image = list(patched_uploaded.keys())[0]
print(f"Uploaded image: {patched_test_image}")

Saving testimage_panda+patch2.png to testimage_panda+patch2.png
Uploaded image: testimage_panda+patch2.png


In [27]:
# Test with the uploaded patched image
predicted_patched_class = classify_image(patched_test_image)

# Print the predicted class name
print(f"Predicted patched class: {predicted_patched_class}")

Predicted patched class: giant panda
